In [490]:
from prometheus_api_client import PrometheusConnect
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Flatten, Reshape

In [2]:
pc = PrometheusConnect(url="http://localhost:9090")

In [450]:
train_start_time = pd.Timestamp('2023-04-08T15:00:00IST')
train_end_time = pd.Timestamp('2023-04-10T04:00:00IST')

validate_start_time = pd.Timestamp('2023-04-10T04:01:00IST')
validate_end_time = pd.Timestamp('2023-04-10T10:00:00IST')

test_start_time = pd.Timestamp('2023-04-10T21:00:00IST')
test_end_time = pd.Timestamp('2023-04-10T22:00:00IST')

# anomaly_test_start_time = pd.Timestamp('2023-04-11T00:00:00IST')
# anomaly_test_end_time = pd.Timestamp('2023-04-11T01:00:00IST')

anomaly_test_start_time = pd.Timestamp('2023-04-11T17:40:00IST')
anomaly_test_end_time = pd.Timestamp('2023-04-11T19:40:00IST')

### Fetch the Metrics

In [457]:
def query_metric(query, start_time, end_time):
    return pc.custom_query_range(query, start_time, end_time, '1m')

In [458]:
# Returns a 2D array of size (num_values, num_nodes)
# num_values depends on the start and end time given when fetching the metric
def extract_values(raw_metric):
    metric = []

    scaler = StandardScaler()
    for data in raw_metric:
        d = np.array(data['values']).T[1]
        scaled_d = scaler.fit_transform(d.reshape((-1, 1))).flatten()
        metric.append(scaled_d)
    
    return np.array(metric, dtype=np.float64).T

In [459]:
def get_metric(query, start_time, end_time):
    raw_metric = query_metric(query, start_time, end_time)
    return extract_values(raw_metric)

### Organize the Metric Values as Features

In [460]:
def get_features(metrics):
    return metrics.transpose((1, 2, 0))

In [461]:
queries = [
    'sum(rate(node_cpu_seconds_total{mode="user"}[10m])) by (node)',
    'node_memory_MemAvailable_bytes',
    'sum(rate(kubelet_http_requests_total[10m])) by (kubernetes_io_hostname)',
    '(sum(rate(node_network_transmit_bytes_total[10m])) by (node))',
    '(sum(rate(node_network_receive_bytes_total[15m])) by (node))',
    'sum(kubelet_running_containers{container_state="running"}) by (kubernetes_io_hostname)',
    'sum(container_processes) by (kubernetes_io_hostname)',
    'node_sockstat_sockets_used',
    'sum(container_sockets) by (kubernetes_io_hostname)',
    'avg(kubelet_http_requests_duration_seconds_sum) by (kubernetes_io_hostname)'
]

In [462]:
m1 = get_metric(queries[0], train_start_time, train_end_time)
m2 = get_metric(queries[1], train_start_time, train_end_time)

In [463]:
m1.shape

(2221, 4)

In [477]:
train_metrics = np.array(
    [get_metric(query, train_start_time, train_end_time) for query in queries]
)

x_train = get_features(train_metrics)

In [478]:
validate_metrics = np.array(
    [get_metric(query, validate_start_time, validate_end_time) for query in queries]
)

x_validate = get_features(validate_metrics)

In [479]:
test_metrics = np.array(
    [get_metric(query, test_start_time, test_end_time) for query in queries]
)

x_test = get_features(test_metrics)

In [480]:
anomaly_test_metrics = np.array(
    [get_metric(query, anomaly_test_start_time, anomaly_test_end_time) for query in queries]
)

x_anomaly_test = get_features(anomaly_test_metrics)

In [481]:
x_train.shape

(2221, 4, 10)

### Model Creation

In [482]:
# Define the input shape
input_shape = (4, 10)

# Define the encoder layers
encoder_input = Input(shape=input_shape)
x1 = Flatten()(encoder_input)

x2 = Dense(32, activation='relu')(x1)

# Define the decoder layers
x3 = Dense(40, activation='linear')(x2)
decoder_output = Reshape(input_shape)(x3)

In [483]:
autoencoder = keras.Model(encoder_input, decoder_output)

In [484]:
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

In [485]:
autoencoder.summary()

Model: "model_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_31 (InputLayer)       [(None, 4, 10)]           0         
                                                                 
 flatten_23 (Flatten)        (None, 40)                0         
                                                                 
 dense_99 (Dense)            (None, 32)                1312      
                                                                 
 dense_100 (Dense)           (None, 40)                1320      
                                                                 
 reshape_23 (Reshape)        (None, 4, 10)             0         
                                                                 
Total params: 2,632
Trainable params: 2,632
Non-trainable params: 0
_________________________________________________________________


In [486]:
autoencoder.fit(x_train, x_train,
                epochs=200,
                shuffle=False,
                validation_data=(x_validate, x_validate))

Epoch 1/200
70/70 [==============================] - 0s 1ms/step - loss: 1.2322 - val_loss: 1.0991
Epoch 2/200
70/70 [==============================] - 0s 597us/step - loss: 0.9237 - val_loss: 0.9839
Epoch 3/200
70/70 [==============================] - 0s 605us/step - loss: 0.7562 - val_loss: 0.9036
Epoch 4/200
70/70 [==============================] - 0s 627us/step - loss: 0.6276 - val_loss: 0.8408
Epoch 5/200
70/70 [==============================] - 0s 584us/step - loss: 0.5354 - val_loss: 0.7878
Epoch 6/200
70/70 [==============================] - 0s 601us/step - loss: 0.4713 - val_loss: 0.7416
Epoch 7/200
70/70 [==============================] - 0s 1ms/step - loss: 0.4220 - val_loss: 0.7008
Epoch 8/200
70/70 [==============================] - 0s 771us/step - loss: 0.3806 - val_loss: 0.6650
Epoch 9/200
70/70 [==============================] - 0s 670us/step - loss: 0.3450 - val_loss: 0.6335
Epoch 10/200
70/70 [==============================] - 0s 708us/step - loss: 0.3146 - val_loss: 

In [487]:
predictions = autoencoder.predict(x_test)
np.mean((predictions - x_test) ** 2, axis=(2, 0))

2/2 [==============================] - 0s 1ms/step


array([0.1745083 , 0.06739435, 0.15302552, 0.07193125])

In [502]:
predictions = autoencoder.predict(x_anomaly_test)
np.mean((predictions - x_anomaly_test) ** 2, axis=(2, 0))

4/4 [==============================] - 0s 500us/step


array([0.29439865, 0.12986821, 0.18996151, 0.09072823])

In [497]:
predictions = autoencoder.predict(x_train)
np.mean((predictions - x_train) ** 2, axis=(2, 0))

70/70 [==============================] - 0s 458us/step


array([0.02057845, 0.02326927, 0.01746798, 0.01508893])

### Detect Attack

In [534]:
nodeNumberToName = {
    0: 'gke-cluster-1-default-pool-a33b274c-9t2g',
    1: 'gke-cluster-1-default-pool-a33b274c-g246',
    2: 'gke-cluster-1-default-pool-a33b274c-hsm7',
    3: 'gke-cluster-1-default-pool-a33b274c-z3f4',
}

def getNodeName(nodeNumber):
    return nodeNumberToName[nodeNumber]

In [538]:
def get_attacked_nodes(x, threshold):
    predictions = autoencoder.predict(x)
    error = np.mean((predictions - x) ** 2, axis=(2, 0))
    print(f'error => {error}')
    nodeNumbers = np.where(error > threshold)[0]
    return [getNodeName(nodeNumber) for nodeNumber in nodeNumbers]

In [541]:
attacked_nodes = get_attacked_nodes(x_anomaly_test, 0.25)

4/4 [==============================] - 0s 3ms/step
error => [0.29439865 0.12986821 0.18996151 0.09072823]


In [542]:
attacked_nodes

['gke-cluster-1-default-pool-a33b274c-9t2g']